<a href="https://colab.research.google.com/github/dmanzanoa/WeatherAgent/blob/main/Weather_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install openai-agents
!pip install requests

In [24]:
import os
os.environ["OPENAI_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxx"  # your new API key here

In [ ]:
import json
import asyncio
import nest_asyncio
from typing_extensions import TypedDict
import requests

from agents import Agent, FunctionTool, RunContextWrapper, function_tool, Runner

nest_asyncio.apply()  # <-- important for Jupyter/Colab

# Tool input: just the city name
class LocationByCity(TypedDict):
    city: str



@function_tool
async def get_weather(location: LocationByCity) -> str:
    """Fetch real-time weather for a given city using OpenWeatherMap API."""
    city = location["city"]
    response = requests.get(
        f"http://api.weatherapi.com/v1/current.json?key=b3c948ca8cb6495c87682750250808&q={city}")
        #f"&appid={OPENWEATHER_API_KEY}&units=metric"
    if response.status_code == 200:
        data = response.json()
        return data
    else:
      return f"Sorry, I couldn't get the weather for {city}."

# Define the agent
weather_agent = Agent(
    name="Weather Agent",
    instructions=("""
    You are a helpful weather assistant. You can only answer questions about the weather of a city.

    If the user asks anything not related to the weather of a specific location, respond with:
    "I'm only able to provide weather information for cities. Please ask me about the weather in a specific place."

    If they ask about a location, call the `get_weather` function with the correct coordinates.
    """),
    tools=[get_weather],
)

# Run test input
async def main():
    result = await Runner.run(weather_agent, input="Give me detailed information about weather in Rome?")
    print(result.final_output)

await main()


Here's the current weather information for Rome:

- **Condition**: Sunny ![Sunny Icon](//cdn.weatherapi.com/weather/64x64/day/113.png)
- **Temperature**: 31.3°C (88.3°F)
- **Feels Like**: 30.0°C (86.1°F)
- **Wind**: 2.2 mph (3.6 kph) from the SW
- **Pressure**: 1019.0 mb (30.09 in)
- **Precipitation**: 0.0 mm
- **Humidity**: 38%
- **Cloud Cover**: 0%
- **Visibility**: 10 km (6 miles)
- **UV Index**: 6.2
- **Gusts**: 2.6 mph (4.1 kph)

Let me know if you need more details!
